In [0]:
#mapeia pastas
bronze_path = "/Volumes/bikestore/default/1_bronze"
silver_path = "/Volumes/bikestore/default/2_silver"
gold_path = "/Volumes/bikestore/default/3_gold"
origem_path = "/Volumes/bikestore/default/4_origem"
destino_path = "/Volumes/bikestore/default/5_destino"

In [0]:
%sql
-- create schema if not exists bikestore.nm_schema; --> cria schema no catalogo "bikstore"
-- drop schema bikestore.nm_schema cascade; --> deleta schema criado

In [0]:
display(dbutils.fs.ls(origem_path))

In [0]:
# ingestao origem --> bronze
df = spark.read.csv(f"{origem_path}/stocks.csv",  header = True, inferSchema = True, sep = ",")
df.show(3)

In [0]:
df.write.mode("overwrite")\
    .format("delta")\
    .option("margeSchema", "true")\
    .save(f"{bronze_path}/stocks")

del(df)

In [0]:
%sql
-- insere resumo da ingestao na tabela de log
-- delete from bikestore.default.tb_ingest_log;

declare or replace variable dt_ingest timestamp;
set variable dt_ingest = (SELECT from_utc_timestamp(CURRENT_TIMESTAMP(), "America/Sao_Paulo"));

insert into bikestore.default.tb_ingest_log values(
  "stocks.csv",
  "bronze",
  "America/Sao_Paulo",
  dt_ingest,
  current_date()
);